## Bibliotecas

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import hamming_loss
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.adapt import MLkNN
import numpy as np

## Carregando os dados

In [2]:
dados = pd.read_csv('Dados/stackoverflow_perguntas.csv')
dados.head()

,Perguntas,Tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js
1,"Gostaria de fazer testes unitários no Node.js,...",node.js
2,Como inverter a ordem com que o jQuery itera u...,jquery
3,Eu tenho uma página onde pretendo utilizar um ...,html
4,Como exibir os dados retornados do FireStore e...,html angular


## Analisando os dados

In [3]:
print(f'Linhas: {dados.shape[0]}')
print(f'Colunas: {dados.shape[1]}')

Linhas: 5408
Colunas: 2


Podemos verificar abaixo que na coluna Tags, temos 37 valores únicos.

In [4]:
print(len(dados['Tags'].unique()))

37


Porém, ao analisar cada um dos valores únicos, notamos existência de alguns valores que podemos tratar.

In [5]:
dados['Tags'].unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

## Tratamento de dados

### Criando uma lista com os tags separados
Abaixo, criamos uma lista de tags únicos existentes no nosso dataframe. Ao executar o código abaixo, podemos observar que há 4 tags diferentes: node.js, jquery, html e angular.

In [6]:
lista_tags = list()

for tags in dados['Tags'].unique():
    for tag in tags.split():
        if tag not in lista_tags:
            lista_tags.append(tag)

print(lista_tags)

['node.js', 'jquery', 'html', 'angular']


### Criando uma função para criar uma coluna para cada tag

Abaixo, criamos uma função para criar colunas de cada tag. Os dados são preeenchidos de acordo com a coluna Tags.

In [7]:
def nova_coluna(lista_tags, dataframe, nome_tags):
    for tag in lista_tags:
        coluna = list()
        for linha_tag in dataframe[nome_tags]:
            if tag in linha_tag:
                coluna.append(1)
            else:
                coluna.append(0)

        dataframe[tag] = coluna

nova_coluna(lista_tags, dados, 'Tags')

dados.head()

,Perguntas,Tags,node.js,jquery,html,angular
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1


### Zip para classificação multilabel

Por fim, juntamos os dados das 4 tags em uma coluna para facilitar a criação e o performance do modelo.

In [8]:
lista_zip = list(zip(dados[lista_tags[0]], 
                dados[lista_tags[1]],
                dados[lista_tags[2]],
                dados[lista_tags[3]],))

dados['todas_tags'] = lista_zip

dados.head()

,Perguntas,Tags,node.js,jquery,html,angular,todas_tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0,"(1, 0, 0, 0)"
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0,"(1, 0, 0, 0)"
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0,"(0, 0, 1, 0)"
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1,"(0, 0, 1, 1)"


## Treinando o modelo

### Separando os dados de treino e teste

Separamos os dados da seguinte forma: a coluna Perguntas será a variável feature e a coluna todas_tags criado anteriormente será a variável target, 20% dos dados serão os dados de teste.

In [9]:
perguntas_treino, perguntas_teste, tags_treino, tags_teste = train_test_split(dados['Perguntas'], 
                                                                              dados['todas_tags'], 
                                                                              test_size = 0.2, 
                                                                              random_state = 123)

### Vetorização dos dados

Por estarmos trabalhando com variável de texto vetorizamos a coluna de perguntas.

In [10]:
# Instanciando o vetorizador.
vetorizador = TfidfVectorizer(max_features = 5000,
                              max_df = 0.85)

# Treinando o vetorizador.
vetorizador.fit(dados['Perguntas'])

# Vetorizando as perguntas de treino e teste.
perguntas_treino_vetorizadas = vetorizador.transform(perguntas_treino)
perguntas_teste_vetorizadas = vetorizador.transform(perguntas_teste)

### Classificação Multilabel: One vs Rest

In [11]:
# Convertendo as tags de treino e teste para array.
tags_treino_array = np.asarray(list(tags_treino))
tags_teste_array = np.asarray(list(tags_teste))

# Instanciando Regressão Logística.
regressao_logistica = LogisticRegression(solver = 'lbfgs')

# Instanciando o classificador.
classificador = OneVsRestClassifier(regressao_logistica)

# Treinando o classificador.
classificador.fit(perguntas_treino_vetorizadas, tags_treino_array)

# Fazendo previsões.
tags_previstas = classificador.predict(perguntas_teste_vetorizadas)

# Calculando a acurácia.
resultado_onevsrest = classificador.score(perguntas_teste_vetorizadas, tags_teste_array)

print(f'Acurácia OneVsRest: {round(resultado_onevsrest, 2) * 100}%')

Acurácia OneVsRest: 42.0%


### Métrica Hamming Loss

Ao calcular o Hamming Loss das previsões realizadas com modelo de classificação OneVsRest, podemos notar que o valor é bem próximo de 0.  

In [12]:
# Calculando o Hamming Loss.
resultado_hamming_loss = hamming_loss(tags_teste_array, tags_previstas)

print(f'Hamming Loss: {round(resultado_hamming_loss, 2)}')

Hamming Loss: 0.19


### Classificação em cadeia

Com a classificação em cadeia (chain), podemos notar houve uma melhoria na acurácia, porém o Hamming Loss aumentou.

In [13]:
# Instanciando o classificador em cadeia.
classificador_cadeia = ClassifierChain(regressao_logistica)

# Treinando o classificador em cadeia.
classificador_cadeia.fit(perguntas_treino_vetorizadas, tags_treino_array)

# Fazendo previsões.
tags_previstas_cadeia = classificador_cadeia.predict(perguntas_teste_vetorizadas)

# Calculando a acurácia.
resultado_cadeia = classificador_cadeia.score(perguntas_teste_vetorizadas, tags_teste_array)

# Calculando o Hamming Loss.
resultado_hamming_loss_cadeia = hamming_loss(tags_teste_array, tags_previstas_cadeia)

print(f'Acurácia ClassifierChain: {round(resultado_cadeia, 2) * 100}%')
print(f'Hamming Loss ClassifierChain: {round(resultado_hamming_loss_cadeia, 2)}')

Acurácia ClassifierChain: 50.0%
Hamming Loss ClassifierChain: 0.21


### Classificação binária utilizando Skmultilearn

Abaixo, foi utilizado Binary Relevance do Skmultilearn que retorna os mesmos resultados do classificador One vs Rest.

In [14]:
# Instanciando o classificador.
classificador_binario = BinaryRelevance(regressao_logistica)

# Treinando o classificador.
classificador_binario.fit(perguntas_treino_vetorizadas, tags_treino_array)

# Fazendo previsões.
tags_previstas_binario = classificador_binario.predict(perguntas_teste_vetorizadas)

# Calculando a acurácia.
resultado_binario = classificador_binario.score(perguntas_teste_vetorizadas, tags_teste_array)

# Calculando o Hamming Loss.
resultado_hamming_loss_binario = hamming_loss(tags_teste_array, tags_previstas_binario)

print(f'Acurácia BinaryRelevance: {round(resultado_binario, 2) * 100}%')
print(f'Hamming Loss BinaryRelevance: {round(resultado_hamming_loss_binario, 2)}')

Acurácia BinaryRelevance: 42.0%
Hamming Loss BinaryRelevance: 0.19


### Classificação MLkNN

Esse algoritmo é especialmente útil em situações onde cada instância pode pertencer a múltiplas classes, como classificação de texto, onde um documento pode ser categorizado em múltiplos tópicos. Sendo assim, treinamos o classificador e o mesmo retornou as métricas baixas em comparação aos demais classificadores.

In [15]:
# Instanciando o classificador.
classificador_mlknn = MLkNN()

# Treinando o classificador.
classificador_mlknn.fit(perguntas_treino_vetorizadas, tags_treino_array)

# Fazendo previsões.
tags_previstas_mlknn = classificador_mlknn.predict(perguntas_teste_vetorizadas)

# Calculando a acurácia.
resultado_mlknn = classificador_mlknn.score(perguntas_teste_vetorizadas, tags_teste_array)

# Calculando o Hamming Loss.
resultado_hamming_loss_mlknn = hamming_loss(tags_teste_array, tags_previstas_mlknn)

print(f'Acurácia MLkNN: {round(resultado_mlknn, 2) * 100}%')
print(f'Hamming Loss MLkNN: {round(resultado_hamming_loss_mlknn, 2)}')

Acurácia MLkNN: 33.0%
Hamming Loss MLkNN: 0.25


### Criando dataframe com os resultados

In [16]:
resultados = pd.DataFrame()
resultados['Perguntas'] = perguntas_teste.values
resultados['Tags_real'] = list(tags_teste)
resultados['Tags_previstas_OneVsRest'] = list(tags_previstas)
resultados['Tags_previstas_ClassifierChain'] = list(tags_previstas_cadeia.toarray())
resultados['Tags_previstas_MLkNN'] = list(tags_previstas_mlknn.toarray())

resultados.head()

,Perguntas,Tags_real,Tags_previstas_OneVsRest,Tags_previstas_ClassifierChain,Tags_previstas_MLkNN
0,estou com conflito entre o CODE e os CODE ...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 0]"
1,Estou fazendo um site que eu sou obrigado a us...,"(0, 0, 1, 0)","[0, 0, 1, 0]","[0.0, 0.0, 1.0, 0.0]","[0, 1, 1, 0]"
2,Recentemente fiz um refactor do meu código par...,"(1, 0, 0, 0)","[1, 0, 0, 0]","[1.0, 0.0, 0.0, 0.0]","[1, 0, 0, 0]"
3,Eu tenho esse código em CODE que passo valore...,"(0, 1, 1, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 1, 0]"
4,"Olá, em minha função tem o evento CODE que de...","(0, 1, 1, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 1, 0]"


## Conclusão

Ao analisar as métricas e Hamming Loss de cada modelo, apesar das melhorias que podem ser implementadas, podemos considerar que o classificador em cadeia pode ser uma solução para classificar as perguntas por ter as métricas medianas em comparação aos demais modelos até o momento.